# BQL Object Model and Pivot Table



## Different use cases

You can request single/multiple securities as well as data items:


```python
# Single security and single data item
request = bql.Request("AAPL US Equity", last)

# Single security and multiple data items (data items in list with default assigned names)
request = bql.Request(AAPL US Equity", [last, high, low])

# Multiple securities and single data item
request = bql.Request(["AAPL US Equity", "IBM US Equity"], {'CLOSE': last})

# Multiple securities and multiple data items (data items in dict with specified names)
request = bql.Request(["AAPL US Equity", "IBM US Equity"], {'CLOSE': last, 'HIGH': high, 'LOW':low})
```

Note that the dates are associated with the data items, not the request.

## Example: Requesting Multiple Securites and Multiple Data Items

<div style="text-align: right"><a href="#BQL-Object-Model-Basics">Return to Top ↑</a></div>

In [23]:
# Import the BQL library
import bql

# Instantiate an object to interface with the BQL service
bq = bql.Service()

# Define the date range for the request
date_range = bq.func.range('2017-06-05','2017-06-09')
date_range2 = bq.func.range('2017-06-05','2017-06-07')

# Define data items for the pricing fields
# Pass the defined date range
last = bq.data.px_last(dates=date_range)
high = bq.data.px_high(dates=date_range)
low = bq.data.px_low(dates=date_range2)

# Generate the request using the security ticker and a list of the data items
request = bql.Request(["AAPL US Equity", "IBM US Equity"], {'CLOSE': last, 'HIGH': high, 'LOW':low})

# Execute the request
response = bq.execute(request)

# Display the response in a data frame
# Use the combined_df function to display 
# the three returned values in a single data frame 
df = bql.combined_df(response)
df

,DATE,CURRENCY,CLOSE,HIGH,LOW
ID,,,,,
AAPL US Equity,2017-06-05,USD,38.482500,38.612500,38.365000
AAPL US Equity,2017-06-06,USD,38.612500,38.952500,38.445000
AAPL US Equity,2017-06-07,USD,38.842500,38.995000,38.620000
AAPL US Equity,2017-06-08,USD,38.747500,38.885000,NaN
AAPL US Equity,2017-06-09,USD,37.245000,38.797500,NaN
IBM US Equity,2017-06-05,USD,145.576545,146.073231,144.881783
IBM US Equity,2017-06-06,USD,145.538339,146.035024,145.184928
IBM US Equity,2017-06-07,USD,144.210661,147.286289,144.038731
IBM US Equity,2017-06-08,USD,145.280444,145.968162,NaN


The results are shown as a dataframe, with securities in the universe (`AAPL US Equity`, `IBM US Equity`) as indices, and the data fields (`CLOSE`, `HIGH`, `LOW`, and the default `DATE` and `CURRENCY`) as the columns.

## Pivot Table

A pivot view returns a reshaped dataframe organized by given index/column values.

<div style="text-align: right"><a href="#BQL-Object-Model-Basics">Return to Top ↑</a></div>

In [24]:
df['SYMBOL'] = df.index
df

,DATE,CURRENCY,CLOSE,HIGH,LOW,SYMBOL
ID,,,,,,
AAPL US Equity,2017-06-05,USD,38.482500,38.612500,38.365000,AAPL US Equity
AAPL US Equity,2017-06-06,USD,38.612500,38.952500,38.445000,AAPL US Equity
AAPL US Equity,2017-06-07,USD,38.842500,38.995000,38.620000,AAPL US Equity
AAPL US Equity,2017-06-08,USD,38.747500,38.885000,NaN,AAPL US Equity
AAPL US Equity,2017-06-09,USD,37.245000,38.797500,NaN,AAPL US Equity
IBM US Equity,2017-06-05,USD,145.576545,146.073231,144.881783,IBM US Equity
IBM US Equity,2017-06-06,USD,145.538339,146.035024,145.184928,IBM US Equity
IBM US Equity,2017-06-07,USD,144.210661,147.286289,144.038731,IBM US Equity
IBM US Equity,2017-06-08,USD,145.280444,145.968162,NaN,IBM US Equity


In [14]:
df1 = df.pivot(index='DATE', columns='SYMBOL', values=['HIGH', 'LOW'])
df1

HIGH                          LOW              
SYMBOL     AAPL US Equity IBM US Equity AAPL US Equity IBM US Equity
DATE                                                                
2017-06-05        38.6125    146.073231         38.365    144.881783
2017-06-06        38.9525    146.035024         38.445    145.184928
2017-06-07        38.9950    147.286289         38.620    144.038731
2017-06-08        38.8850    145.968162            NaN           NaN
2017-06-09        38.7975    147.348374            NaN           NaN

In [17]:
df1.loc['2017-06-08', 'HIGH']['AAPL US Equity']

38.885

## Example D: Requesting Data for Multiple Securities <a name="D"></a>
Now that we know how to pull multiple data items across a range of dates for a single security, let's look at a few different ways to define a universe of securities:

- Define a list of securities. For example `securities = ['MSFT US Equity', 'IBM US Equity', 'AMZN US Equity']`
- Call a BQL function that returns a list of securities, such as the `bq.univ.members()` function, which returns the members of an index.

In this example, we'll use bq.univ.members() to pass the members of the INDU Index.

### Example Code

In [18]:
# Import the BQL library
import bql

# Instantiate an object to interface with the BQL service
bq = bql.Service()

# Define a universe of the members of the INDU Index
indu = bq.univ.members("INDU Index")

# Define the date range for the request
date_range = bq.func.range('2017-06-05','2017-06-09')

# Define data items for the pricing fields
# Pass the defined date range
last = bq.data.px_last(dates=date_range)
high = bq.data.px_high(dates=date_range)
low = bq.data.px_low(dates=date_range)

# Generate the request using the security universe and a list of the data items
request = bql.Request(indu, [last, high, low])

# Execute the request
response = bq.execute(request)

# Display the response in a DataFrame
# Use the combined_df function to display 
# the three returned values in a single DataFrame
# To verify the output, use tail(3) to show the last three rows of the response
bql.combined_df(response).tail(3)

,DATE,CURRENCY,"PX_LAST(dates=RANGE(2017-06-05,2017-06-09))","PX_HIGH(dates=RANGE(2017-06-05,2017-06-09))","PX_LOW(dates=RANGE(2017-06-05,2017-06-09))"
ID,,,,,
NKE UN Equity,2017-06-07,USD,53.23,53.36,52.75
NKE UN Equity,2017-06-08,USD,53.20,54.14,53.15
NKE UN Equity,2017-06-09,USD,53.46,53.47,52.81


<div style="text-align: right"><a href="#BQL-Object-Model-Basics">Return to Top ↑</a></div>

## Related Resources <a name="related"></a>
- <a href = "./BQL String Interface Basics.ipynb">BQL String Interface Basics</a>
- <a href = "exampleroot/53815e8850a54b4187fb0d1335ee39d5/BQL Field Function and Universe Search.ipynb">BQL Field, Function, and Universe Search</a>
- <a href = "exampleroot/9032931110d6467fa8f1938424453f06/_BQL Quick Reference Guide.ipynb">BQL Quick Reference Guide</a>
- <a href = "exampleroot/f2d2cebf9dac40699094b3caa8ad35ce/BQL Fundamental Data.ipynb">BQL Fundamental Data</a>
